In [178]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [179]:
import os

import json

import llama_cpp
import torch
import pandas as pd

from dotenv import load_dotenv
from openai import OpenAI

from curverag import utils
from curverag.curverag import CurveRAG, DEFAULT_ENTITY_TYPES, DEFAULT_GLINER_MODEL, DEFAULT_SENTENCE_TRANSFORMER_MODEL
from curverag.graph import KnowledgeGraph
from curverag.atth.kg_dataset import KGDataset
from curverag.atth.models.hyperbolic import AttH
from curverag.eval import evaluation, queries, context, expected_output

load_dotenv() 

True

# Explore dataset

In [180]:
with open('../datasets/2WikiMultihopQA/new/train.json', 'rb') as f:
    train = json.load(f)

with open('../datasets/2WikiMultihopQA/new/dev.json', 'rb') as f:
    dev = json.load(f)

with open('../datasets/2WikiMultihopQA/new/test.json', 'rb') as f:
    test = json.load(f)

In [181]:
len(dev)

12576

In [182]:
dev[0].keys()

dict_keys(['_id', 'type', 'question', 'context', 'entity_ids', 'supporting_facts', 'evidences', 'answer', 'evidences_id', 'answer_id'])

In [183]:
train[0].keys()

dict_keys(['_id', 'type', 'question', 'context', 'entity_ids', 'supporting_facts', 'evidences', 'answer', 'evidences_id', 'answer_id'])

In [184]:
train[0]['question']

'Are director of film Move (1970 Film) and director of film Méditerranée (1963 Film) from the same country?'

In [185]:
train[0]['context'][:3]

[['Stuart Rosenberg',
  ['Stuart Rosenberg (August 11, 1927 – March 15, 2007) was an American film and television director whose motion pictures include "Cool Hand Luke" (1967), "Voyage of the Damned" (1976), "The Amityville Horror" (1979), and "The Pope of Greenwich Village" (1984).',
   'He was noted for his work with actor Paul Newman.']],
 ['Méditerranée (1963 film)',
  ['Méditerranée is a 1963 French experimental film directed by Jean-Daniel Pollet with assistance from Volker Schlöndorff.',
   'It was written by Philippe Sollers and produced by Barbet Schroeder, with music by Antione Duhamel.',
   'The 45 minute film is cited as one of Pollet\'s most influential films, which according to Jonathan Rosenbaum directly influenced Jean-Luc Goddard\'s "Contempt", released later the same year.',
   'Footage for the film was shot around the Mediterranean, including at a Greek temple, a Sicilian garden, the sea, and also features a fisherman, a bullfighter, and a girl on an operating table

In [186]:
train[0]['supporting_facts']

[['Move (1970 film)', 0],
 ['Méditerranée (1963 film)', 0],
 ['Stuart Rosenberg', 0],
 ['Jean-Daniel Pollet', 0]]

In [187]:
train[0]['evidences']

[['Move (1970 film)', 'director', 'Stuart Rosenberg'],
 ['Méditerranée (1963 film)', 'director', 'Jean-Daniel Pollet'],
 ['Stuart Rosenberg', 'country of citizenship', 'American'],
 ['Jean-Daniel Pollet', 'country of citizenship', 'French']]

In [188]:
train[0]['answer']

'no'

# Run eval

## Train Model

In [257]:
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

In [262]:
entity_types = ['people', 'locations', 'entities', 'movies', 'directors']
rag = CurveRAG(
    openai_client=client,
    entity_types=entity_types,
)

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

2025-09-03 22:29:33,093 INFO     Use pytorch device_name: mps
2025-09-03 22:29:33,093 INFO     Use pytorch device_name: mps
2025-09-03 22:29:33,093 INFO     Use pytorch device_name: mps
2025-09-03 22:29:33,093 INFO     Use pytorch device_name: mps
2025-09-03 22:29:33,093 INFO     Use pytorch device_name: mps
2025-09-03 22:29:33,093 INFO     Use pytorch device_name: mps
2025-09-03 22:29:33,093 INFO     Use pytorch device_name: mps
2025-09-03 22:29:33,093 INFO     Use pytorch device_name: mps
2025-09-03 22:29:33,093 INFO     Use pytorch device_name: mps
2025-09-03 22:29:33,093 INFO     Use pytorch device_name: mps
2025-09-03 22:29:33,093 INFO     Use pytorch device_name: mps
2025-09-03 22:29:33,099 INFO     Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-09-03 22:29:33,099 INFO     Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-09-03 22:29:33,099 INFO     Load pretrained SentenceTransformer: all-MiniLM-L6-v2
2025-09-03 22:29:33,099 INFO     Load pretrained Sentence

In [263]:
new_docs = []
for d in dev:
    d_ = str(d['context']).replace('[', '').replace(']', '. ').replace('\'', '')
    new_docs.append(d_)

In [264]:
len(new_docs)

12576

In [ ]:
dataset_size = 3
rag.fit(new_docs[:dataset_size], dataset_name='movies_10')

In [ ]:
rag.save(path="./models/250902_model_dev_10_1/model.pkl")

In [ ]:
rag = CurveRAG.load(
    "./models/250824_model_dev_1/model.pkl",
    openai_client=client  # or llm=llm, outlines_llm=outlines_llm
)

In [267]:
len(rag.graph.nodes)

35

In [270]:
rag.dataset.n_entities

16

## Eval Model

In [278]:
dataset_size=10
ems, f1, qs, preds, answers, graphs = evaluation(rag, dataset_size=dataset_size)

query: Who is the mother of the director of film Polish-Russian War (Film)?
['Polish-Russian War', 'mother', 'director', 'film', 'War', 'Film']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-09-03 22:50:53,695 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:50:53,695 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:50:53,695 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:50:53,695 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:50:53,695 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:50:53,695 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:50:53,695 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:50:53,695 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:50:53,695 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:50:53,695 INFO     HTTP Request:

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-09-03 22:51:08,884 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:51:08,884 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:51:08,884 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:51:08,884 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:51:08,884 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:51:08,884 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:51:08,884 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:51:08,884 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:51:08,884 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 22:51:08,884 INFO     HTTP Request:

query: Who is the maternal grandfather of Antiochus X Eusebes?
['grandfather', 'Antiochus', 'X', 'Eusebes']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Found no embedding indx for entities, doing non KG-RAG result


In [279]:
len(preds)

10

In [280]:
ems, f1

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [281]:
sum(f1) / len(f1)

0.1

In [80]:
for f, q, p, a, g, r in zip(f1, qs, preds, answers, graphs, dev[:dataset_size]):
    print('Q: ', q)
    print('P: ', p)
    print('A: ', a)
    print('F:', f)
    print('S:', r['supporting_facts'], r['evidences'])
    print('G: ', g)
    print('-------------------------------------------')

Q:  Who is the mother of the director of film Polish-Russian War (Film)?
P:  N/A
A:  ['Małgorzata Braunek', 'Małgorzata Braunek']
F: 0.0
S: [['Polish-Russian War (film)', 1], ['Xawery Żuławski', 2]] [['Polish-Russian War', 'director', 'Xawery Żuławski'], ['Xawery Żuławski', 'mother', 'Małgorzata Braunek']]
G:  KnowledgeGraph Overview
  There are 43 entities and 25 relationships in this graph.

Entities in the graph:
  • 'Viktor Yeliseyev':
      The entity has the following description: Russian general, orchestra conductor, and music teacher. Russian general, orchestra conductor, and music teacher.
      It can also be referred to as: Viktor Petrovich Yeliseyev, Viktor Yeliseyev
      It has the following additional information: Born June 9, 1950., Director of the Ministry of the Interior Ensemble.
  • 'Polish-Russian War (film)':
      The entity has the following description: 2009 Polish film directed by Xawery Żuławski.
      It has the following additional information: Based on the

In [29]:
for f, q, p, a, g, r in zip(f1, qs, preds, answers, graphs, dev[:dataset_size]):
    print('Q: ', q)
    print('P: ', p)
    print('A: ', a)
    print('F:', f)
    print('S:', r['supporting_facts'], r['evidences'])
    print('G: ', g)
    print('-------------------------------------------')

Q:  Who is the mother of the director of film Polish-Russian War (Film)?
P:  Dorota Masłowska
A:  ['Małgorzata Braunek', 'Małgorzata Braunek']
F: 0.0
S: [['Polish-Russian War (film)', 1], ['Xawery Żuławski', 2]] [['Polish-Russian War', 'director', 'Xawery Żuławski'], ['Xawery Żuławski', 'mother', 'Małgorzata Braunek']]
G:  KnowledgeGraph Overview
  There are 71 entities and 27 relationships in this graph.

Entities in the graph:
  • 'Polish-Russian War (film)':
      The entity has the following description: 2009 Polish film directed by Xawery Żuławski. 2009 Polish film directed by Xawery Żuławski.
      It can also be referred to as: Polish-Russian War (film)
      It has the following additional information: Based on the novel Polish-Russian War under the white-red flag by Dorota Masłowska.
  • 'Xawery Żuławski':
      The entity has the following description: Polish film director born December 22, 1971. Polish film director born December 22, 1971.
      It can also be referred to as

In [ ]:
sum(f1) / len(f1)

In [292]:
h_ems, h_f1, h_qs, h_preds, h_answers, h_graphs = evaluation(rag, dataset_size=10, model_traversal='hyperbolic')

query: Who is the mother of the director of film Polish-Russian War (Film)?
['Polish-Russian War', 'mother', 'director', 'film', 'War', 'Film']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

embedding_idx [[]]
getting edges for nodes
EE: []
EE: []
finished getting edges for nodes


2025-09-03 23:28:33,499 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:33,499 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:33,499 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:33,499 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:33,499 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:33,499 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:33,499 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:33,499 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:33,499 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:33,499 INFO     HTTP Request:

query: Which film came out first, Blind Shaft or The Mask Of Fu Manchu?
['Blind Shaft', 'The Mask Of Fu Manchu', 'film', 'came', 'Blind', 'Shaft', 'Mask', 'Fu', 'Manchu']


...

embedding_idx [[]]
getting edges for nodes
EE: []
finished getting edges for nodes


2025-09-03 23:28:40,311 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:40,311 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:40,311 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:40,311 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:40,311 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:40,311 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:40,311 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:40,311 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:40,311 INFO     HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-09-03 23:28:40,311 INFO     HTTP Request:

query: Who is the maternal grandfather of Antiochus X Eusebes?
['grandfather', 'Antiochus', 'X', 'Eusebes']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Found no embedding indx for entities, doing non KG-RAG result


In [287]:
h_ems, h_f1

([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
 [0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0])

In [288]:
sum(h_f1) / len(h_f1)

0.2